In [17]:
import pandas as pd
import backtrader as bt

# 1. 读取Excel文件
df = pd.read_excel('D:/code/quant_code/行情数据/btc_usdt_candlesticks.xlsx')

# 2. 检查列名
print("列名:", df.columns)

# 3. 格式化数据
# 将时间戳转换为datetime格式
df['date'] = pd.to_datetime(df['ts'], unit='ms')

# 4. 创建Backtrader数据源
class PandasData(bt.feeds.PandasData):
    lines = ('close', 'high', 'open',)
    # params = (
    #     ('datetime', 'date'),  # 时间列
    #     ('open', 'o'),         # 开盘价
    #     ('high', 'h'),         # 最高价
    #     ('low', 'l'),          # 最低价
    #     ('close', 'c'),        # 收盘价
    #     ('volume', 'vol'),     # 成交量
    #     ('volCcy', 'volCcy'),  # 货币成交量
    #     ('volCcyQuote', 'volCcyQuote'),  # 货币成交量（报价）
    #     ('confirm', 'confirm'), # 确认
    # )

# 创建回测引擎
cerebro = bt.Cerebro()

# 不再设置索引，直接使用原始数据
data = PandasData(dataname=df)
cerebro.adddata(data)

# 添加你的策略
class MyStrategy(bt.Strategy):
    def __init__(self):
        self.ma = bt.indicators.SimpleMovingAverage(self.data.close, period=20)

    def next(self):
        if self.data.close[0] > self.ma[0]:
            self.buy()
        elif self.data.close[0] < self.ma[0]:
            self.sell()

cerebro.addstrategy(MyStrategy)

# 运行回测
try:
    cerebro.run()
except Exception as e:
    print("回测失败:", e)
# 设置图表大小
cerebro.plot(size=(12, 8))  # 设置图表尺寸
# 可选：绘制结果
cerebro.plot()


列名: Index(['ts', 'open', 'high', 'low', 'close', 'vol', 'volCcy', 'volCcyQuote',
       'confirm', 'date'],
      dtype='object')
回测失败: 'int' object has no attribute 'to_pydatetime'


[]

In [11]:
import pandas as pd
import backtrader as bt

# 1. 读取Excel文件
df = pd.read_excel('D:/code/quant_code/行情数据/btc_usdt_candlesticks.xlsx', index_col='date')

# 2.创建策略
class MyStrategt(bt.Strategy):
    
    params = (
        ('myparam',20),
        ('exitbars',5)
    )
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        self.dataclose = self.datas[0].close
        # 跟踪订单
        self.order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 订单提交和成交当前不做处理
            return 
        # 检查订单是否成交
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('购买完成, %.2f' % order.executed.price)
            elif order.issell():
                self.log('出售完成, %.2f' % order.executed.price)
            
            self.bar_executed = len(self)
            
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单取消')
            
        # 记录没有挂起的订单   
        self.order = None
        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        
        # 检查如果有挂起的订单，不能再发起一个
        if self.order:
            return 
        
        if self.dataclose[0] < self.dataclose[-1]:
            # 连续两天下跌就买入
            if self.dataclose[-1] < self.dataclose[-2]:
                self.log('买！，%.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            # 已经在市场5天，5天后卖掉
            if(len(self) >= (self.bar_executed + self.params.exitbars)):
                self.log('卖出！，%.2f' % self.dataclose[0])
                self.order = self.sell()
        

# 3.创建大脑
cerebro = bt.Cerebro()
# 4.添加策略
cerebro.addstrategy(MyStrategt, myparam=20, exitbars=7)
# 5.喂数据
data = bt.feeds.PandasData(dataname = df)
cerebro.adddata(data)

# 券商佣金 0.1%
cerebro.broker.setcommission(commission=0.01)
cerebro.broker.setcash(500000)

print('初始资产：%.2f' % cerebro.broker.getvalue())

cerebro.run()

cerebro.plot()

print('结束资产：%.2f' % cerebro.broker.getvalue())


初始资产：500000.00
2023-08-22, Close, 26145.90
2023-08-23, Close, 26001.90
2023-08-23, 买！，26001.90
2023-08-24, 购买完成, 26001.90
2023-08-24, Close, 25932.30
2023-08-24, 买！，25932.30
2023-08-25, 购买完成, 25932.30
2023-08-25, Close, 26055.40
2023-08-26, Close, 26123.20
2023-08-27, Close, 26104.50
2023-08-28, Close, 27420.50
2023-08-29, Close, 27204.00
2023-08-30, Close, 26923.50
2023-08-30, 买！，26923.50
2023-08-31, 购买完成, 26923.50
2023-08-31, Close, 25810.40
2023-08-31, 买！，25810.40
2023-09-01, 购买完成, 25810.40
2023-09-01, Close, 25898.20
2023-09-02, Close, 25872.10
2023-09-03, Close, 25815.20
2023-09-03, 买！，25815.20
2023-09-04, 购买完成, 25815.20
2023-09-04, Close, 25758.70
2023-09-04, 买！，25758.70
2023-09-05, 购买完成, 25758.60
2023-09-05, Close, 25592.10
2023-09-05, 买！，25592.10
2023-09-06, 购买完成, 25591.00
2023-09-06, Close, 25776.10
2023-09-07, Close, 25846.30
2023-09-08, Close, 25893.60
2023-09-09, Close, 25774.50
2023-09-10, Close, 25114.00
2023-09-10, 买！，25114.00
2023-09-11, 购买完成, 25112.90
2023-09-11, Close

ValueError: Axis limits cannot be NaN or Inf

In [19]:
#创建一个策略
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 20),
    )
 #记录功能
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 引用到close line
        self.dataclose = self.datas[0].close

        # 跟踪订单状态以及买卖价格和佣金
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # 增加移动均线
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 订单状态为提交和接受，不做处理
            return

        # 检查订单是否成交
        # 注意，没有足够现金的话，订单会被拒绝。
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # 记录当前处理的close值
        self.log('Close, %.2f' % self.dataclose[0])

        # 订单是否
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # 大于均线就买
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # 小于均线卖卖卖！
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


if __name__ == '__main__':
    cerebro = bt.Cerebro()

    # 增加一个策略
    cerebro.addstrategy(TestStrategy)

    #获取数据
    df = pd.read_excel('D:/code/quant_code/行情数据/btc_usdt_candlesticks.xlsx', index_col='date')
    # start_date = datetime(2020, 9, 30)  # 回测开始时间
    # end_date = datetime(2021, 9, 30)  # 回测结束时间
    data = bt.feeds.PandasData(dataname=df)  # 加载数据
    cerebro.adddata(data)  # 将数据传入回测系统

    cerebro.broker.setcash(100000.0)
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0)
     # Add a FixedSize sizer according to the stake 每次买卖的股数量
    cerebro.addsizer(bt.sizers.FixedSize, stake=70)



    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.plot()

Starting Portfolio Value: 100000.00
2023-09-10, Close, 25114.00
2023-09-11, Close, 26209.20
2023-09-11, BUY CREATE, 26209.20
2023-09-12, Order Canceled/Margin/Rejected
2023-09-12, Close, 26246.10
2023-09-12, BUY CREATE, 26246.10
2023-09-13, Order Canceled/Margin/Rejected
2023-09-13, Close, 26696.40
2023-09-13, BUY CREATE, 26696.40
2023-09-14, Order Canceled/Margin/Rejected
2023-09-14, Close, 26320.90
2023-09-14, BUY CREATE, 26320.90
2023-09-15, Order Canceled/Margin/Rejected
2023-09-15, Close, 26510.40
2023-09-15, BUY CREATE, 26510.40
2023-09-16, Order Canceled/Margin/Rejected
2023-09-16, Close, 26548.70
2023-09-16, BUY CREATE, 26548.70
2023-09-17, Order Canceled/Margin/Rejected
2023-09-17, Close, 27271.80
2023-09-17, BUY CREATE, 27271.80
2023-09-18, Order Canceled/Margin/Rejected
2023-09-18, Close, 27425.10
2023-09-18, BUY CREATE, 27425.10
2023-09-19, Order Canceled/Margin/Rejected
2023-09-19, Close, 27180.20
2023-09-19, BUY CREATE, 27180.20
2023-09-20, Order Canceled/Margin/Rejected


<IPython.core.display.Javascript object>